<a href="https://www.kaggle.com/code/sailikhitarage/maize-mutant-classification-using-vgg16?scriptVersionId=105286810" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Importing necessary Libraries**

In [ ]:
import os
import numpy as np
import pandas as pd
import tqdm as tqdm
import cv2 as cv
import keras
from keras.models import Sequential
from keras.layers import Conv2D , MaxPooling2D ,GlobalAveragePooling2D ,Flatten , Dense , Dropout , BatchNormalization 
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
tf.keras.applications.VGG16
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.models import Model 

**Data Preprocessing**

converting categorical labels to numeric codes

In [ ]:
encoder = OneHotEncoder()     #LabelEncoder encode labels with value between 0 and n_classes-1
encoder.fit([[0],[1],[2],[3]])   

**Loading the data**

Resizing the image based on input dimension required for the model

In [ ]:
input_path = []
label = []

In [ ]:
data = []
paths = []
result = []
fpath = []
for r,d,f in os.walk(r"../input/corn-or-maize-leaf-disease-dataset/data/Blight"):
    for file in f:
        if '.jpg' in file:
            paths.append(os.path.join(r,file))
for path in paths:
    img = Image.open(path)
    img = img.resize((224,224))
    img = np.array(img)
    if(img.shape == (224,224,3)):
        data.append(img)
        label.append(0)
        input_path.append(os.path.join("Blight","Blight",path))
        result.append(encoder.transform([[0]]).toarray())
print(len(paths))

In [ ]:
paths = []
for r,d,f in os.walk(r"../input/corn-or-maize-leaf-disease-dataset/data/Common_Rust"):
    for file in f:
        if '.jpg' in file:
            paths.append(os.path.join(r,file))
for path in paths:
    img = Image.open(path)
    img = img.resize((224,224))
    img = np.array(img)
    if(img.shape == (224,224,3)):
        data.append(img)
        label.append(1)
        input_path.append(os.path.join("Common_Rust","Common_Rust",path))
        result.append(encoder.transform([[1]]).toarray())
print(len(paths))

In [ ]:
paths = []
for r,d,f in os.walk(r"../input/corn-or-maize-leaf-disease-dataset/data/Gray_Leaf_Spot"):
    for file in f:
        if '.jpg' in file:
            paths.append(os.path.join(r,file))
for path in paths:
    img = Image.open(path)
    img = img.resize((224,224))
    img = np.array(img)
    if(img.shape == (224,224,3)):
        data.append(img)
        label.append(2)
        input_path.append(os.path.join("Gray_Leaf_Spot","Gray_Leaf_Spot",path))
        result.append(encoder.transform([[2]]).toarray())
print(len(paths))
print(len(input_path))

In [ ]:
paths = []
for r,d,f in os.walk(r"../input/corn-or-maize-leaf-disease-dataset/data/Healthy"):
    for file in f:
        if '.jpg' in file:
            paths.append(os.path.join(r,file))
for path in paths:
    img = Image.open(path)
    img = img.resize((224,224))
    img = np.array(img)
    if(img.shape == (224,224,3)):
        data.append(img)
        label.append(3)
        input_path.append(os.path.join("Healthy","Healthy",path))
        result.append(encoder.transform([[3]]).toarray())
print(len(paths))

In [ ]:
df = pd.DataFrame()
df['images'] = input_path
df['label'] = label
df=df.sample(frac=1).reset_index(drop=True)
df.head()

In [ ]:
pd.unique(df['label'])

In [ ]:
import seaborn as sns
sns.countplot(df['label'])

In [ ]:
print("Total no.of images are :",len(result))

In [ ]:
len(data)

In [ ]:
len(label)

In [ ]:
result = np.array(result)
result = result.reshape(-1,4)

In [ ]:
result.shape

In [ ]:
data = np.array(data)
data.shape

**Displaying random images**

In [ ]:
%matplotlib inline
plt.figure(figsize=(10, 10))
for i in range(6):
    plt.subplot(1, 6, i+1)
    plt.imshow(data[i], cmap="gray")
    plt.axis('off')
plt.show()

**Splitting data into training and testing data**

In [ ]:
x_train , x_test , y_train , y_test = train_test_split(data , result , test_size = 0.25 , shuffle = True , random_state = 100)

In [ ]:
print("Shape of an image in x_train : ",x_train[0].shape)
print("Shape of an image in x_test : ",x_test[0].shape)

In [ ]:
x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
print("x_train Shape : ", x_train.shape) 
print("x_test Shape : ", x_test.shape)
print("y_train Shape: ", y_train.shape) 
print("y_test Shape: ", y_test.shape)

**Pre-Trained VGG16 model**

In [ ]:
modelVGG = VGG16(include_top = False,weights = 'imagenet', classifier_activation = 'Softmax',input_shape = (224, 224, 3))

In [ ]:
for layer in modelVGG.layers:
    layer.trainable = False

In [ ]:
num_classes = 4
for (i,layer) in enumerate(modelVGG.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)
    def lw(bottom_model,num_classes):
        top_model = bottom_model.output
        top_model = GlobalAveragePooling2D()(top_model)
        top_model = Dense(4096,activation='relu')(top_model)
        top_model = Dropout(0.625, name='dropout_1')(top_model)
        top_model = Dense(4096,activation='relu')(top_model)
        #top_model = Dropout(0.625, name='dropout_2')(top_model)
        top_model = Dense(512,activation='relu')(top_model)
        top_model = Dense(4,activation='softmax')(top_model)
        return top_model

In [ ]:
num_classes = 4
FC_Head = lw(modelVGG , num_classes)
model = Model(inputs = modelVGG.input,outputs = FC_Head)

**Summary of the network layers**

In [ ]:
print(model.summary())

In [ ]:
# summarize filters in each convolutional layer
#from keras.applications.vgg16 import VGG16
#from matplotlib import pyplot
# load the model
#model = VGG16()
# summarize filter shapes
for layer in model.layers:
# check for convolutional layer
    if 'conv' not in layer.name:
        continue
    # get filter weights
    filters, biases = layer.get_weights()
    print(layer.name, filters.shape)


In [ ]:
# normalize filter values to 0-1 so we can visualize them
f_min, f_max = filters.min(), filters.max()
filters = (filters - f_min) / (f_max - f_min)

In [ ]:
import matplotlib.pyplot as plt
f = plt.figure(figsize=(16,16))
nof_filters, ix = 4, 1
for i in range(nof_filters):
    f = filters[:, :, :, i]
    # plot each channel separately
    for j in range(3):
        ax = plt.subplot(nof_filters, 3, ix)
        ax.set_xticks([])
        ax.set_yticks([])
        # plot filter channel in grayscale
        plt.imshow(f[:, :, j], cmap='gray')
        ix += 1
# show the figure
plt.show()

**Compiling the VGG16 model**

In [ ]:
model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

**Fitting the Model**

In [ ]:
history = model.fit(x_train , y_train , epochs = 40, validation_data = (x_test , y_test), verbose = 1)

In [ ]:
%matplotlib inline
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

In [ ]:
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('EPOCHS')
plt.ylabel('LOSS')
plt.legend(loc=0)
plt.figure()

plt.show()

In [ ]:
plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.xlabel('EPOCHS')
plt.ylabel('ACCURACY')

plt.figure()

plt.show()

In [ ]:
print('Test accuracy:', model.evaluate(x_test, y_test))

In [ ]:
def names(number):
    if number==0:
        return "It's a leaf with disease BLIGHT"
    elif number==1:
        return "It's a leaf with disease Common Rust"
    elif number==2:
        return "It's a leaf with disease Gray_Leaf_Spot"
    elif number==3:
        return "It's a Healthy leaf"

**Testing the model**

In [ ]:
from matplotlib.pyplot import imshow
def Prediction(img):
    #img = Image.open(r"../input/corn-or-maize-leaf-disease-dataset/data/Blight/Corn_Blight (1010).JPG")
    x = np.array(img.resize((224,224)))
    x = x.reshape(1,224,224,3)
    res = model.predict_on_batch(x)
    classification = np.where(res == np.amax(res))[1][0]
    imshow(img)
    print(str(res[0][classification]*100) + '% Confidence ' + names(classification))

In [ ]:
img = Image.open(r"../input/corn-or-maize-leaf-disease-dataset/data/Blight/Corn_Blight (1010).JPG")
Prediction(img)

Testing with other random images

In [ ]:
img = Image.open(r"../input/leafmutant/Leaf_disease_testdataset/Blight/Bim6.jpg")
Prediction(img)

In [ ]:
img = Image.open(r"../input/corn-or-maize-leaf-disease-dataset/data/Gray_Leaf_Spot/Corn_Gray_Spot (103).JPG")
Prediction(img)

In [ ]:
img = Image.open(r"../input/leafmutant/Leaf_disease_testdataset/GrayLeafSpot/Gim1.jpg")
Prediction(img)

In [ ]:
img = Image.open(r"../input/leafmutant/Leaf_disease_testdataset/GrayLeafSpot/Gim7.jpg")
Prediction(img)

In [ ]:
img = Image.open(r"../input/leafmutant/Leaf_disease_testdataset/CommonRust/CRim10.jpg")
Prediction(img)

In [ ]:
img = Image.open(r"../input/corn-or-maize-leaf-disease-dataset/data/Healthy/Corn_Health (1034).jpg")
Prediction(img)

In [ ]:
img = Image.open(r"../input/leafmutant/Leaf_disease_testdataset/Healthy/im8.jpg")
Prediction(img)